In [4]:
import gymnasium as gym
import numpy as np
import pygame
from gymnasium import spaces
import random
import gymnasium as gym
from gymnasium import spaces
from gymnasium.envs.registration import register
from gymnasium.utils.env_checker import check_env
import numpy as np
from typing import Optional, Union
import math

from gym.utils import seeding
import matplotlib.pyplot as plt
import matplotlib.image as mpimg  # Ajoutez cette ligne
import random
import os

from matplotlib import patches, animation
from matplotlib.transforms import Affine2D

class AcasEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 8}  # Ralentir l'animation à 15 FPS

    act_to_angle = [0, np.radians(-1.5), np.radians(+1.5), np.radians(-3.), np.radians(+3.)]

    def __init__(self, 
                 save_states=False,
                 render_mode=None,
                 airplanes=None,  
                 epsilon=153,
                 max_time_steps=200, step=0):
        self.last_a = 0
        self.epsilon = epsilon
        self.save_states = save_states
        self.states_history = []
        self.commands_history = []
        self.min_dist = 0
        self.max_time_steps = max_time_steps
        self.render_mode = render_mode
        self.screen = None
        self.clock = None
        self.window_size = (1200, 800)  # Agrandir la fenêtre PyGame

        

        self.first_step = True
        self.done = False
        self.info = {}
        self.current_time_step = 0

        self.action_space = spaces.Discrete(5)

        self.observation_space = spaces.Box(
           low=np.array([-1e6, 0, 0, -np.pi, -np.pi, 0]),
           high=np.array([1e6, 300, 300, np.pi, np.pi, 4]),
           dtype=np.float32
        )

        if airplanes is None:
            self.airplanes = self.smart_random_init()
        else:
            self.airplanes = airplanes
        
        self.own = self.airplanes[0]
        self.int = self.airplanes[1]
        self.rho = np.sqrt((self.own.x - self.int.x) ** 2 + (self.own.y - self.int.y) ** 2)

    def update_relations(self):
        self.relative_distances = np.array([[np.hypot(own.x-intruder.x, own.y-intruder.y) for intruder in self.airplanes] for own in self.airplanes])
        for i in range(len(self.airplanes)):
            self.relative_distances[i,i] = np.inf
        self.nearest_intruder_index = self.relative_distances.argmin(axis=-1)
        self.relative_angles = np.array([[rad_mod(np.arctan2(intruder.y-own.y, intruder.x-own.x)) for intruder in self.airplanes] for own in self.airplanes])
        self.relative_heads = np.array([[rad_mod(intruder.head-own.head) for intruder in self.airplanes] for own in self.airplanes])
        self.rho = np.array([self.relative_distances[i, self.nearest_intruder_index[i]] for i in range(len(self.airplanes))])
        self.theta = np.array([self.relative_angles[i, self.nearest_intruder_index[i]] for i in range(len(self.airplanes))])
        self.psi = np.array([self.relative_heads[i, self.nearest_intruder_index[i]] for i in range(len(self.airplanes))])
        self.v_int = np.array([self.airplanes[self.nearest_intruder_index[i]].speed for i in range(len(self.airplanes))])
        self.min_dist = self.rho.min()

    def _get_obs(self):
        own = self.airplanes[0]
        intruder = self.airplanes[1]

        rho = np.clip(self.rho[0], -1e4, 1e4)
        own_speed = np.clip(own.speed, 0, 300)
        intruder_speed = np.clip(intruder.speed, 0, 300)
        theta = np.clip(self.theta[0], -np.pi, np.pi)
        psi = np.clip(self.psi[0], -np.pi, np.pi)
        last_a = np.clip(self.last_a, 0, 4)

        obs = np.array([rho, own_speed, intruder_speed, theta, psi, last_a], dtype=np.float32)
        return obs

    def _get_info(self):
        own = self.airplanes[0]
        intruder = self.airplanes[1]
        
        info = {
        'own_position': (own.x, own.y),
         'intruder_position': (intruder.x, intruder.y),
         'distance_to_intruder': self.rho[0]
         }

        return info
    
    def smart_random_init(self, total_time=120):
        airplanes = []
        interest_time = random.randrange(40, total_time - 40)

        head = np.random.uniform(-np.pi, np.pi)
        speed = np.random.uniform(100, 300)
        x_t, y_t = 0, 0
        x_0 = x_t - (speed * interest_time * np.cos(head))
        y_0 = y_t - (speed * interest_time * np.sin(head))
        airplanes.append(Airplane(x=x_0, y=y_0, head=head, speed=speed, name="own"))

        head = np.random.uniform(-np.pi, np.pi)
        speed = np.random.uniform(100, 300)
        x_t, y_t = 0, 0
        x_0 = x_t - (speed * interest_time * np.cos(head))
        y_0 = y_t - (speed * interest_time * np.sin(head))
        airplanes.append(Airplane(x=x_0, y=y_0, head=head, speed=speed, name="int"))

        return airplanes

    def reset(self, *, seed: Optional[int] = None, options: Optional[dict] = None):
        super().reset(seed=seed)

        self.airplanes = self.smart_random_init()
        self.update_relations()
        
        observation = self._get_obs()
        info = {}
        self.current_time_step = 0
        self.first_step = True  # Réinitialisation du drapeau pour chaque nouvel épisode
        if self.render_mode == "human":
            self.render()
        
        self.done = False

        return observation, info

    def step(self, action):
        own = self.airplanes[0]
        intruder = self.airplanes[1]

        own.head = rad_mod(own.head + self.act_to_angle[action])
        
        # Réduire l'ampleur du déplacement à chaque étape pour ralentir le mouvement
        own.x += np.cos(own.head) * own.speed   # Réduction de la vitesse par un facteur de 0.05
        own.y += np.sin(own.head) * own.speed 

        intruder.x += np.cos(intruder.head) * intruder.speed 
        intruder.y += np.sin(intruder.head) * intruder.speed 

        print(np.degrees(intruder.head))
        print(np.degrees(own.head))

        self.update_relations()

        self.current_time_step += 1
        
        reward = 0

        if action == 0:
            reward += 0.0001
        elif ((self.last_a == 1 and action == 3) or (self.last_a == 2 and action == 4)):
            reward -= 0.009
        elif ((self.last_a == 1 or self.last_a == 3) and (action == 2 or action == 4)):
            reward -= 0.01
        elif ((self.last_a == 2 or self.last_a == 4) and (action == 3 or action == 1)):
            reward -= 0.01
        elif self.rho[-1] < self.epsilon:
            reward = -1

        terminated = (self.rho[0] < self.epsilon)
        truncated = (self.current_time_step == self.max_time_steps)
        
        self.last_a = action
        obs = self._get_obs()
        info = self._get_info()
 
        return obs, reward, terminated, truncated, info

    def render(self, mode="human"):
        if self.screen is None:
            pygame.init()
            self.screen = pygame.display.set_mode(self.window_size)
            pygame.display.set_caption("ACAS XU Simulation")
            self.clock = pygame.time.Clock()
            airplane_image_path = "/d/wabouir/acas-v2/img/airplane.png"
            self.airplane_image = pygame.image.load(airplane_image_path).convert_alpha()
            self.airplane_image = pygame.transform.scale(self.airplane_image, (50,50))

        if self.first_step:  # Nettoyage de l'écran uniquement au début de chaque épisode
            self.screen.fill((255, 255, 255))  # Remplir l'écran avec du blanc
            self.first_step = False

        own = self.airplanes[0]
        intruder = self.airplanes[1]

        scale = 0.05  # Ajustement de l'échelle pour mieux adapter la fenêtre
        own_pos = (int(own.x * scale + self.window_size[0] / 2), int(own.y * scale + self.window_size[1] / 2))
        intruder_pos = (int(intruder.x * scale + self.window_size[0] / 2), int(intruder.y * scale + self.window_size[1] / 2))

        rotated_img_own=pygame.transform.rotate(self.airplane_image, -np.degrees(own.head)-90)
        rotated_img_int = pygame.transform.rotate(self.airplane_image, -np.degrees(intruder.head)-90)

        rect_own = rotated_img_own.get_rect(center=own_pos)
        rect_int = rotated_img_int.get_rect(center=intruder_pos)

        self.screen.blit(rotated_img_own, rect_own.topleft)
        self.screen.blit(rotated_img_int, rect_int.topleft)

        
        #pygame.draw.circle(self.screen, (0, 0, 255), own_pos, 10)
        #pygame.draw.circle(self.screen, (255, 0, 0), intruder_pos, 10)

        pygame.display.flip()
        self.screen.fill((255, 255, 255))
        self.clock.tick(self.metadata["render_fps"])

    def close(self):
        if self.screen is not None:
            pygame.quit()
            self.screen = None

# Fonction pour normaliser les angles
def rad_mod(angle):
    """return angle between -pi and +pi"""
    return ((angle + np.pi) % (np.pi*2)) - np.pi

# Classe Airplane
class Airplane():
    def __init__(self, x=0.0, y=0.0, head=0.0, speed=1.0, name='airplane'):
        self.x = x   
        self.y = y   
        self.head = head  # in rad
        self.speed = speed  # distance per unit of time (on heading direction)
        self.name = name
        self.last_a = 0

    def __str__(self):
        return f"x: {self.x}, y: {self.y}, head: {self.head}, speed: {self.speed}"

# Créer une instance de l'environnement
env = AcasEnv(render_mode="human")

# Réinitialiser l'environnement
obs, info = env.reset()

# Définir le nombre maximal d'étapes
num_steps = 200

# Boucle sur chaque étape du jeu
for step in range(num_steps):
    # Prendre l'action 0 (aller tout droit)
    action = 1
    obs, reward, terminated, truncated, info = env.step(action)

    # Rendre l'environnement pour visualiser
    env.render()

    # Vérifier si l'épisode est terminé ou tronqué
    if terminated or truncated:
        print(f"Episode terminé à l'étape {step + 1}")
        break

# Fermer l'environnement pour libérer les ressources
env.close()


72.68360260637448
-13.468714599687026
72.68360260637448
-14.968714599687035
72.68360260637448
-16.468714599687043
72.68360260637448
-17.96871459968705
72.68360260637448
-19.46871459968706
72.68360260637448
-20.96871459968707
72.68360260637448
-22.46871459968708
72.68360260637448
-23.968714599687086
72.68360260637448
-25.468714599687093
72.68360260637448
-26.968714599687104
72.68360260637448
-28.46871459968711
72.68360260637448
-29.968714599687118
72.68360260637448
-31.46871459968713
72.68360260637448
-32.96871459968714
72.68360260637448
-34.46871459968715
72.68360260637448
-35.968714599687154
72.68360260637448
-37.46871459968716
72.68360260637448
-38.96871459968717
72.68360260637448
-40.46871459968718
72.68360260637448
-41.96871459968719
72.68360260637448
-43.468714599687196
72.68360260637448
-44.9687145996872
72.68360260637448
-46.46871459968721
72.68360260637448
-47.968714599687225
72.68360260637448
-49.46871459968723
72.68360260637448
-50.96871459968724
72.68360260637448
-52.4687145